In [1]:
import pandas as pd
import pickle
import json

In [2]:
import requests
import json
import ast
from tqdm import tqdm
import time
import pickle
from yuno1.ui import base as uBase
from yuno1.search import base as sBase
import PIL
from io import BytesIO

HTML(value='<style>\n\n  .searchTerm {\n    border: 3px solid #916BBF;\n    border-radius: 3px;\n\n  }\n\n  .s…

In [3]:
# with open('yuno/info_base (1).pkl', 'rb') as f:
#     data = pickle.load(f)

In [4]:
# with open('data/anime_data.json', encoding='utf-8') as f:
#     anime_data = json.load(f)

In [5]:
# anime_data['rows'][0]['json']

In [6]:
# temp = json.loads(anime_data['rows'][0]['json'])

In [7]:
# temp.keys()

In [8]:
anime_info_dict= {}

In [9]:
data = pd.read_csv('data/anime_data.csv')

In [10]:
data.keys()

Index(['id', 'json'], dtype='object')

In [11]:
data['id'][0]

1

In [12]:
json.loads(data['json'][0])['id']

1

In [13]:
def get_data(index):
    json_data = json.loads(data['json'][index])
    uid = json_data['id']
    names = list(json_data["title"].values())
    img_url = json_data["coverImage"]["large"]
    
    
    synopsis = json_data["description"]
    anilist_url = json_data["siteUrl"]
    mal_url = "https://myanimelist.net/anime/"+str(uid)
    anime_info_dict[sBase.AnimeUid(uid)] = uBase.AnimeInfo(names,img_url,synopsis,anilist_url,mal_url)

In [14]:
for index in range(len(data['id'])):
    get_data(index)

In [15]:
len(anime_info_dict)

17773

In [16]:
anime_info_dict[sBase.AnimeUid(2904)]

AnimeInfo(names=['コードギアス 反逆のルルーシュ R2', 'Code Geass: Hangyaku no Lelouch R2', 'Code Geass: Lelouch of the Rebellion R2'], synopsis='A year has passed since "The Black Rebellion" and the remaining Black Knights have vanished into the shadows, their leader and figurehead, Zero, executed by the Britannian Empire. Area 11 is once more squirming under the Emperor\'s oppressive heel as the Britannian armies concentrate their attacks on the European front. But for the Britannians living in Area 11, life is back to normal. On one such normal day, a Britannian student, skipping his classes in the Ashford Academy, sneaks out to gamble on his chess play. But unknown to this young man, several forces are eying him from the shadows, for soon, he will experience a shocking encounter with his own obscured past, and the masked rebel mastermind Zero will return.', anilist_url='https://anilist.co/anime/2904', mal_url='https://myanimelist.net/anime/2904')

In [17]:
from yuno.preprocessing.filter import *

In [18]:
anime_infos = {int:AnimeInfo}

In [19]:
# for id in anime_info_dict.keys():
#     anime_infos[id].uid = id
#     anime_infos[id].names = anime_info_dict[id].names

In [20]:
len(json.loads(data['json'][1])["characters"]["edges"])

25

In [21]:
gender = json.loads(data['json'][1])["characters"]["edges"][0]['node']['gender']
name = json.loads(data['json'][1])["characters"]["edges"][0]['node']['name']
char_name = name['first'] + " " + name['last']
print(gender,name)
print(char_name)

Male {'first': 'Spike', 'last': 'Spiegel', 'native': 'スパイク・スピーゲル', 'alternative': ['Spike Spike']}
Spike Spiegel


In [22]:
char = Character([char_name,name['alternative']],Gender.MALE)

In [23]:
print(char)

Character(names=['Spike Spiegel', ['Spike Spike']], gender=<Gender.MALE: '[MALE_CHAR]'>)


In [24]:
from typing import List

In [25]:
json.loads(data['json'][1])['coverImage']['medium']

'https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/b5-Zs2cbrglTu67.png'

In [26]:
gender_dict = {}
gender_dict['MALE'] = Gender.MALE
gender_dict['FEMALE'] = Gender.FEMALE
gender_dict['NO'] = Gender.CHAR
gender_dict['NON-BINARY'] = Gender.CHAR

In [33]:
from crawler import *

In [30]:
def get_ani_data(index):
    json_data = json.loads(data['json'][index])
    uid = json_data['idMal']
    titles = list(json_data["title"].values())
    #Remove None value and end periods
    titles = remove_none_value_list(titles)
    titles = remove_end_period_list(titles)
    
    characters = []
    character_json = json_data["characters"]["edges"]
    for char_id in range(len(character_json)):
        full_name = None
        try:
            char_name = Character.extract_name(character_json,char_id)
            # char_name = character_json[char_id]['node']['name']
            first_name = char_name['first']
            last_name = char_name['last']
            full_name = (first_name + " " + last_name).strip()
        except:
            pass
        
        try:
            char_gender = character_json[char_id]['node']['gender']
        except:
            char_gender = 'No'
            
        if str(char_gender).upper() not in gender_dict or None:
            gend = gender_dict['NO']
        else:
            gend = gender_dict[str(char_gender).upper()]
            
        name_lst = [alt for alt in char_name['alternative']]
        
        # if full_name is not None:
        #     name_lst.append(full_name)
        if first_name is not None:
            if len(first_name)>1:
                name_lst.append(first_name)
        if len(name_lst)<1:
            continue
        char = Character(name_lst, gend)
        characters.append(char)
    if uid is None:
        return
    # anime_infos[sBase.AnimeUid(uid)] = AnimeInfo(uid,titles,characters)
    # anime_infos[sBase.AnimeUid(uid)] = {'uid':uid,'names':titles,'characters':characters}
    print(f"Added anime id {index}")

In [31]:
get_ani_data(0)

Added anime id 0


In [45]:
for index in range(len(data['id'])):
    get_ani_data(index)

Added anime id 0
Added anime id 1
Added anime id 2
Added anime id 3
Added anime id 4
Added anime id 5
Added anime id 6
Added anime id 7
Added anime id 8
Added anime id 9
Added anime id 10
Added anime id 11
Added anime id 12
Added anime id 13
Added anime id 14
Added anime id 15
Added anime id 16
Added anime id 17
Added anime id 18
Added anime id 19
Added anime id 20
Added anime id 21
Added anime id 22
Added anime id 23
Added anime id 24
Added anime id 25
Added anime id 26
Added anime id 27
Added anime id 28
Added anime id 29
Added anime id 30
Added anime id 31
Added anime id 32
Added anime id 33
Added anime id 34
Added anime id 35
Added anime id 36
Added anime id 37
Added anime id 38
Added anime id 39
Added anime id 40
Added anime id 41
Added anime id 42
Added anime id 43
Added anime id 44
Added anime id 45
Added anime id 46
Added anime id 47
Added anime id 48
Added anime id 49
Added anime id 50
Added anime id 51
Added anime id 52
Added anime id 53
Added anime id 54
Added anime id 55
Ad

In [41]:
with open('data/anime_info.pickle', 'wb') as f:
    pickle.dump(anime_infos, f)

In [30]:
print(type(anime_infos))
print(anime_infos[34096].names)

<class 'dict'>
['銀魂。', 'Gintama', 'Gintama Season 5']


In [46]:
anime_infos[34599]

AnimeInfo(uid=34599, names=['メイドインアビス', 'Made in Abyss', 'Made in Abyss'], characters=[Character(names=['Rico', 'Riko'], gender=<Gender.FEMALE: '[FEMALE_CHAR]'>), Character(names=['Regu', 'The Treasure of the Deep'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Lord of Dawn', 'The Novel', 'Delver of the Abyss'], gender=<Gender.CHAR: '[CHAR]'>), Character(names=['The Immovable Sovereign', 'The Immovable', 'Ozen', 'Ouzen'], gender=<Gender.CHAR: '[CHAR]'>), Character(names=['Lord of Annihilation', 'The Annihilator', 'Lyza'], gender=<Gender.CHAR: '[CHAR]'>), Character(names=['Nat'], gender=<Gender.CHAR: '[CHAR]'>), Character(names=['Shiggy'], gender=<Gender.CHAR: '[CHAR]'>), Character(names=['Marulk'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Haborg', 'Habo-san', 'Hablog'], gender=<Gender.CHAR: '[CHAR]'>), Character(names=['Kiyui', 'Kiui', 'Kiwi'], gender=<Gender.CHAR: '[CHAR]'>), Character(names=['Leader (リーダ)', 'Jiruo'], gender=<Gender.CHAR: '[CHAR]'>), Characte